# A primer for PRiMeR

In [1]:
%load_ext autoreload

## Imports

In [2]:
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from pandas_plink import read_plink

import torch
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
%autoreload 2
from primer.preprocess import gaussianize
from primer.gwas import GWAS
from primer.mrbased import UVMRbased, MVMRbased
from primer.model import PRiMeR
from primer.lrm import LRM

## Load Data

In [5]:
# set path to data
risk_factor_file = "../data/risk_factors.parquet.gzip"
genotype_file = "../data/genotypes"
covariate_file = "../data/covariates.parquet.gzip"
sumstats_file = "../data/target_disease.sumstat"

# followup outcome
# this is not required to train PRiMeR, may remain unobserved in applications
# here the simulated values are available and are used for validation
outcome_file = "../data/dis_outcome.parquet.gzip"

In [6]:
# risk factor matrix
dfRF = pd.read_parquet(risk_factor_file)
print('Risk factor shape:', dfRF.shape)
dfRF.head()

Risk factor shape: (50000, 30)


,Risk_factor_0,Risk_factor_1,Risk_factor_2,Risk_factor_3,Risk_factor_4,Risk_factor_5,Risk_factor_6,Risk_factor_7,Risk_factor_8,Risk_factor_9,...,Risk_factor_20,Risk_factor_21,Risk_factor_22,Risk_factor_23,Risk_factor_24,Risk_factor_25,Risk_factor_26,Risk_factor_27,Risk_factor_28,Risk_factor_29
0,0.720357,2.063063,-0.380008,0.717654,0.586087,1.183065,0.651498,-0.527197,-0.477083,1.944030,...,0.466053,-0.715269,-0.463753,-1.154618,0.356706,-0.255902,0.662552,-0.943494,1.277293,1.459237
1,1.085589,-2.059031,-0.481375,-1.907853,0.401829,-0.702551,0.821321,-2.809480,-0.748050,-1.270696,...,-1.760603,-0.015225,-0.069053,0.169353,1.168955,0.647636,0.471785,0.150448,-0.297059,1.343909
2,-0.510135,0.594982,0.015300,0.207112,0.647155,0.102647,0.241790,-0.656938,0.331739,0.816445,...,-1.639291,0.292549,0.763102,0.185799,0.650815,0.009687,0.556082,0.800248,0.208552,0.214982
3,-0.097017,-1.014121,-0.168655,0.623520,0.317475,-0.429457,1.546509,0.169287,0.709109,-0.760504,...,-0.100758,0.772184,-1.351660,-2.096998,-1.091636,-0.308198,0.125971,-1.554938,0.404935,0.665339
4,-0.165761,-0.141431,-0.768626,1.090526,-0.957056,-0.059247,-1.342707,-0.839267,1.941332,-0.184904,...,-0.524927,-1.116354,1.551500,-0.301459,-0.729179,-1.059694,-1.624618,-1.407638,-0.166433,-0.567031


In [7]:
# covariates matrix
dfC = pd.read_parquet(covariate_file)
print('Covariate shape:', dfC.shape)
dfC.head()

Covariate shape: (50000, 2)


,sex,age
0,0.0,53.0
1,1.0,49.0
2,1.0,69.0
3,0.0,58.0
4,0.0,77.0


In [8]:
# To obtain the genetic variants:
# 1. Run GWAS on every trait in the risk factor cohort
# 2. Combine all the GWAS results into a single joint summary statistics file
# 3. Harmonize the joint summary statistics with our outcome sumstats
# 4. Perform LD clumping to obtain a set of independent genetic variants and save ad plink binary file
# 5. Use pandas_plink to read plink binary file and convert to pandas Dataframe

bim, fam, bed = read_plink(str(genotype_file))
dfG = 2 - pd.DataFrame(bed.compute().T, columns=bim["snp"].values) # pandas_plink reads 2 as reference allele => convert to minor allele count
print("Genotype shape:", dfG.shape)
dfG.head()

Mapping files:   0%|                                                                                                                            | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_2186740/1726984066.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  bim, fam, bed = read_plink(str(genotype_file))
Mapping files:  33%|██████████████████████████████████████▋                                                                             | 1/3 [00:00<00:00,  4.70it/s]/tmp/ipykernel_2186740/1726984066.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  bim, fam, bed = read_plink(str(genotype_file))
Mapping files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.36it/s]


Genotype shape: (50000, 755)


,sid1,sid2,sid3,sid4,sid5,sid6,sid7,sid8,sid9,sid10,...,sid746,sid747,sid748,sid749,sid750,sid751,sid752,sid753,sid754,sid755
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [9]:
# 
dfss_outcome = pd.read_csv(sumstats_file, sep="\t", index_col=0)
print('Sumstat shape:', dfss_outcome.shape)
dfss_outcome.head()

Sumstat shape: (755, 3)


,beta,beta_ste,pv
sid1,0.009139,0.011262,0.417067
sid2,-0.004640,0.008037,0.563756
sid3,0.001874,0.008754,0.830482
sid4,0.022847,0.009466,0.015790
sid5,-0.001009,0.014778,0.945590


In [10]:
# followup outcome
dfo = pd.read_parquet(outcome_file)
print('Followup outcome shape:', dfo.shape)
print('Note: this is not required to train PRiMeR, may remain unobserved in applications. Here the simulated values are available and are used for validation.')
dfo.head()

Followup outcome shape: (50000, 1)
Note: this is not required to train PRiMeR, may remain unobserved in applications. Here the simulated values are available and are used for validation.


,0
0,0.046186
1,-0.503982
2,-1.643379
3,-0.474728
4,-0.424113


In [11]:
# check indices match
assert (dfC.index==dfRF.index).all(), 'outch'
assert (dfC.index==dfo.index).all(), 'outch'
assert (dfC.index==dfG.index).all(), 'outch'
assert (dfss_outcome.index==dfG.columns).all(), 'outch'

## Preprocess Data

In [12]:
# Impute and gaussianize Risk Factors
dfRF = pd.DataFrame(
    gaussianize(SimpleImputer(strategy="mean").fit_transform(dfRF)),
    index=dfRF.index,
    columns=dfRF.columns,
)

# Imputing Genos
dfG = pd.DataFrame(
    SimpleImputer(strategy="mean").fit_transform(dfG),
    index=dfG.index,
    columns=dfG.columns,
)

# Standardize covariates and add intercept
dfC[dfC.columns] = StandardScaler().fit_transform(dfC)
dfC['intercept'] = 1

# Filter rare variants
s_before = dfG.shape[1]
af = 0.5 * dfG.mean(0)
maf = np.minimum(af, 1 - af)
Ikeep = list(maf >= 0.01)
dfG = dfG.loc[:, Ikeep]
s_after = dfG.shape[1]
dfss_outcome = dfss_outcome.loc[Ikeep, :]

# Converting to float64
dfRF = dfRF.astype(np.float64)
dfG = dfG.astype(np.float64)
dfC = dfC.astype(np.float64)

## Split in train and test

In [13]:
# define train / val splits
idxs_train, idxs_val = train_test_split(
    np.arange(dfRF.shape[0]), test_size=0.2, random_state=42, shuffle=True
)

# train: used to train PRiMeR
dfRF_train = dfRF.iloc[idxs_train].copy()
dfG_train = dfG.iloc[idxs_train].copy()
dfC_train = dfC.iloc[idxs_train].copy()

# train outcome: only used to train the LRM
dfo_train = dfo.iloc[idxs_train].copy()

# val: used to validate predictor vs followup outcome
dfRF_val = dfRF.iloc[idxs_val].copy()
dfo_val = dfo.iloc[idxs_val].copy()

# standardize the  riskfactors on the training set
scaler = StandardScaler()
dfRF_train[dfRF_train.columns] = scaler.fit_transform(dfRF_train)
dfRF_val[dfRF_val.columns] = scaler.transform(dfRF_val)

## Run the models

### MR based models

In [14]:
# compute gwas summary stats for risk factors (for mr based models)
gwas_rf = GWAS(dfRF_train.values, dfC_train.values)
gwas_rf.process(dfG_train.values)
beta_rf = gwas_rf.getBetaSNP()
pval_rf = gwas_rf.getPv()

In [15]:
# Prediction model based on univate Mendelian randomization
uvmr = UVMRbased(pv_iv=5e-8, min_ivs=5, fwer_exp=0.05)
dfres_uvmr = uvmr.train(
    beta_rf,
    pval_rf,
    dfss_outcome["beta"].values,
    dfss_outcome["beta_ste"].values,
    dfRF_train.columns,
)

pred_uvmr = uvmr.predict(dfRF_val.values)

In [16]:
# Prediction model based on multivariable Mendelian randomization
mvmr = MVMRbased()
dfres_mvmr = mvmr.train(
    beta_rf,
    dfss_outcome["beta"].values,
    dfss_outcome["beta_ste"].values,
    dfRF_train.columns,
)

pred_mvmr = mvmr.predict(dfRF_val.values)

### Probabilistic Risk Modeling using Mendelian Randomization (PRiMeR)

In [17]:
RF = torch.tensor(dfRF_train.values, dtype=torch.float32)
G = torch.tensor(dfG_train.values, dtype=torch.float32)
F = torch.tensor(dfC_train.values, dtype=torch.float32)

beta_o = torch.tensor(dfss_outcome["beta"].values, dtype=torch.float32).reshape(-1, 1)
beta_se_o = torch.tensor(dfss_outcome["beta_ste"].values, dtype=torch.float32).reshape(
    -1, 1
)

E_v = torch.tensor(dfRF_val.values, dtype=torch.float32)
o_v = torch.tensor(dfo_val.values, dtype=torch.float32)
# RF.shape, G.shape, F.shape, beta_o.shape, beta_se_o.shape, E_t.shape, o_t.shape

In [18]:
# fit primer lin
primer_lin = PRiMeR(RF, G, beta_o, beta_se_o, F=F, lin=True)
primer_lin.optimize_lbfgs()
primerlin_pred = primer_lin.predict(E_v).detach().numpy()

In [19]:
# full nonlinear primer
primer = PRiMeR(RF, G, beta_o, beta_se_o, F=F, lin=False)
primer.log_ve.data[:] = primer_lin.log_ve.data[:]
primer.log_vn.data[:] = primer_lin.log_vn.data[:]
dmr_history = primer.optimize_sgd(
    epochs=1000, lr=1e-2, E=E_v, e_real=o_v, rocauc=False
)
primer_pred = primer.predict(E_v).detach().numpy()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:34<00:00, 28.64it/s]


## Model with same functional form as PRiMeR but with access to followup data

In [20]:
o_train = torch.tensor(dfo_train.values, dtype=torch.float32)
lrm = LRM(RF, o_train, "elu", "regression")
lrm_history = lrm.optimize_sgd(epochs=1000, lr=1e-2, E_val=None, o_val=None)
lrm_pred = lrm.predict(E_v).detach().numpy()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 95.11it/s]


## Compute metrics and print table

In [21]:
df_results = pd.DataFrame(
    data=np.concatenate(
        [pred_uvmr, pred_mvmr, primerlin_pred, primer_pred, lrm_pred], axis=1
    ),
    columns=["UVMR-based", "MVMR-based", "PRiMeR-LIN", "PRiMeR", "LRM (gold standard)"],
)

df_results.apply(lambda x: spearmanr(x, dfo_val.values)[0]).to_frame()

,0
UVMR-based,0.662906
MVMR-based,0.701448
PRiMeR-LIN,0.701476
PRiMeR,0.731347
LRM (gold standard),0.818394
